In [1]:
import lasio
import pandas as pd


In [362]:
reference_list = pd.read_excel(r"C:\Users\srorta\Desktop\measured_shear_files.xlsx")
reference_df = pd.DataFrame(reference_list)
#display(reference_df)

for index,roworig in reference_df.iterrows():
    print(roworig[0])
    print(roworig[1])
    las = lasio.read(fr"{roworig[0]}") #change las file path
    orig_las_df = las.df()
    reset_index_df = orig_las_df.reset_index()
    display(reset_index_df)
    
    cds_df = pd.read_excel(fr"{roworig[1]}") #change deviation survey (from WIRE)
    display(cds_df)
    well_name = cds_df["Well Name"][0]
    print(well_name)

    tvddepthlist = []
    mddepthlist =[]
    latitude =[]
    longitude=[]
    easting = []
    northing = []
    for i,row in reset_index_df.iterrows():
        #find high and low indices inside the CDS table
        mddepthlist.append(round(row['DEPTH'],4))
        searchindex = 0
        while searchindex < len(cds_df) and cds_df["tvdrt"][searchindex] < row['DEPTH']:
            searchindex += 1
        lowindex = max(searchindex -1, 0)
        highindex = min(searchindex, len(cds_df)-1)
        print(lowindex)
        print(highindex)
        print(row['DEPTH'])

        #handle edge cases
        if lowindex == highindex:
            md = None
            tvd = None
            lat = None
            long = None
            east = None
            north = None
            if (lowindex ==0):
                tvd = cds_df["tvdrt"][0]
                md = cds_df["measuredDepth"][0]
                lat = cds_df["Latitude"][0]
                long = cds_df["Longitude"][0]
                east = cds_df["easting"][0]
                north = cds_df["northing"][0]
            else:
                searchindex = len(cds_df)-1
                while cds_df["tvdrt"][searchindex] is None and searchindex >0:
                    searchindex -= 1
                tvd = cds_df["tvdrt"][searchindex]
                md = cds_df['measuredDepth'][searchindex]
                lat = cds_df['Latitude'][searchindex]
                long = cds_df['Longitude'][searchindex]
                east = cds_df["easting"][searchindex]
                north = cds_df["northing"][searchindex]
            delta = tvd-md
            tvdshiftdepth = row['DEPTH'] + delta
            tvdshiftdepth = round(tvdshiftdepth, 4)
            latitude.append(lat)
            longitude.append(long)
            easting.append(east)
            northing.append(north)
            tvddepthlist.append(tvdshiftdepth)
            print(tvdshiftdepth)
        # handle interpolation 
        if lowindex != highindex:
            lowtvddepth = cds_df["tvdrt"][lowindex]
            hightvddepth = cds_df["tvdrt"][highindex]
            lowmddepth = cds_df["measuredDepth"][lowindex]
            highmddepth = cds_df["measuredDepth"][highindex]
            lowlat = cds_df["Latitude"][lowindex]
            highlat = cds_df["Latitude"][highindex]
            lowlong = cds_df["Longitude"][lowindex]
            highlong = cds_df["Longitude"][highindex]  
            loweast = cds_df["easting"][lowindex]
            higheast = cds_df["easting"][highindex]
            lownorth = cds_df["northing"][lowindex]
            highnorth = cds_df["northing"][highindex]
            #if any of the above values are none, abort mission
            #if any(val is None for val in [lowCoreDepth, highCoreDepth, lowLogDepth, highLogDepth]):

            deltatvddepth = hightvddepth - lowtvddepth
            deltamddepth = highmddepth - lowmddepth 
            deltalat = highlat - lowlat
            deltalong = highlong - lowlong
            deltaeast = higheast - loweast
            deltanorth = highnorth - lownorth
            factor = (row['DEPTH'] - lowmddepth)/ deltamddepth
            tvdshiftdepth = factor * deltatvddepth + lowtvddepth
            lat = factor * deltalat + lowlat
            long = factor * deltalong + lowlong
            east = factor * deltaeast + loweast
            north = factor * deltanorth + lownorth
            tvdshiftdepth =round(tvdshiftdepth, 4)
            tvddepthlist.append(tvdshiftdepth)
            latitude.append(lat)
            longitude.append(long)
            easting.append(east)
            northing.append(north)
            print(tvdshiftdepth)


    depth_view_data = {"MD (m)": mddepthlist, "Tvd (m)": tvddepthlist, "Latitude (DD)WGS84": latitude, "Longitude (DD) WGS84": longitude, "Easting (m) WGS84 UTM Zone 21N": easting, "Northing (m) WGS84 UTM Zone 21N": northing}
    depth_view_df = pd.DataFrame(depth_view_data)
    depth_view_df

    depth_view_df["MD-TVD Delta (m)"] = depth_view_df["MD (m)"]-depth_view_df["Tvd (m)"]
    depth_view_df["Reference Elevation (m)"] = roworig[2] #change value based on well
    depth_view_df["Water Depth (m)"] = roworig[3] #change value based on well
    depth_view_df["Depth bml (m)"] = depth_view_df["Tvd (m)"]- (depth_view_df["Water Depth (m)"]+depth_view_df["Reference Elevation (m)"])
    depth_view_df['TVDss(m)'] = depth_view_df["Tvd (m)"] - depth_view_df["Reference Elevation (m)"]
    #if roworig[4] == 'Measured':
        #depth_view_df["VS (m/s)"] = reset_index_df["VS"] #for wells with VS_XW change from "VS" to "VS_XW"
    #else: 
        #depth_view_df["VS (m/s)"] = reset_index_df["VS_XW"]
    depth_view_df["Well Name"] = well_name
    display(depth_view_df)


    depth_view_df.to_excel(f"c:\\users\\srorta\\Desktop\\tvd_shear_final\\{well_name}_{roworig[4]}.xlsx") #change predicted and measured based on log curve